In [32]:
import pandas as pd
import networkx as nx

In [34]:
data = pd.read_csv('data/Denmark/Merged.csv')

,Unnamed: 0,typeid_x,afstemningid,aktørid,opdateringsdato_x,id_y,typeid_y,navn,opdateringsdato_y,startdato,slutdato,party
0,0,1,6922,221,2019-01-15T16:31:11.32,221,5,Pia Adelsteen,2024-12-11T12:49:52.567,2010-10-06T00:00:00,NaN,Dansk Folkeparti
1,1,1,6921,221,2019-01-16T10:45:19.98,221,5,Pia Adelsteen,2024-12-11T12:49:52.567,2010-10-06T00:00:00,NaN,Dansk Folkeparti
2,2,2,6923,221,2019-01-22T13:02:09.41,221,5,Pia Adelsteen,2024-12-11T12:49:52.567,2010-10-06T00:00:00,NaN,Dansk Folkeparti
3,3,1,6924,221,2019-01-24T10:02:15.453,221,5,Pia Adelsteen,2024-12-11T12:49:52.567,2010-10-06T00:00:00,NaN,Dansk Folkeparti
4,4,1,6925,221,2019-01-24T10:03:04.13,221,5,Pia Adelsteen,2024-12-11T12:49:52.567,2010-10-06T00:00:00,NaN,Dansk Folkeparti
...,...,...,...,...,...,...,...,...,...,...,...,...
302093,302093,1,10110,21044,2025-01-16T17:00:16.147,21044,5,Lasse Haugaard Pedersen,2025-02-03T09:39:35.91,2024-12-01T00:00:00,NaN,Socialdemokratiet
302094,302094,1,10111,21044,2025-01-23T10:01:05.52,21044,5,Lasse Haugaard Pedersen,2025-02-03T09:39:35.91,2024-12-01T00:00:00,NaN,Socialdemokratiet
302095,302095,1,10112,21044,2025-01-23T10:01:33.953,21044,5,Lasse Haugaard Pedersen,2025-02-03T09:39:35.91,2024-12-01T00:00:00,NaN,Socialdemokratiet
302096,302096,1,10113,21044,2025-01-23T10:02:11.817,21044,5,Lasse Haugaard Pedersen,2025-02-03T09:39:35.91,2024-12-01T00:00:00,NaN,Socialdemokratiet


In [65]:
#dict, key = id, values = (afstemningid, typeid)
# dict of all people with all their votes (voting id and what they voted)

stuff = {}          #<<-------------- need a better name
for idx, row in data.iterrows():
    if row['aktørid'] not in stuff:
        stuff[row['aktørid']] = [(row['afstemningid'],row['typeid_x'])]
    else:
        stuff[row['aktørid']].append((row['afstemningid'],row['typeid_x']))


In [63]:
# using the dict, find the threshold
# go through all people that are not themselves, check if they have identical votes and divide it by the one that voted the most
def calc_aggrement_thres(dict):
    agrements = []

    for i in dict:
        for j in dict:
            if i != j:
                common = set(dict.get(i)) & set(dict.get(j))
                percentage = len(common) / max(len(dict.get(i)),len(dict.get(j)))
                agrements.append(percentage)
                
    avg = sum(agrements) / len(agrements)
    return avg


    


In [61]:
res = calc_aggrement_thres(stuff)

In [62]:
res

0.10865498508598848

In [64]:
#graph testing
navne = set({})
party = set({})
for i in data['navn']:
    navne.add(i)
for i in data['party']:
    party.add(i)

G = nx.Graph()
for idx, row in data.iterrows():
   G.add_node(row['aktørid'], navn=row['navn'])